In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,8373
2,Huelva,Confirmados PDIA 14 días,2580
3,Huelva,Tasa PDIA 14 días,"505,1464239353256"
4,Huelva,Confirmados PDIA 7 días,1051
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,2
629,Municipio de Huelva sin especificar,Total Confirmados,104
630,Municipio de Huelva sin especificar,Curados,28


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  8373.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2319.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 439 personas en los últimos 7 días 

Un positivo PCR cada 195 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,8373.0,1051.0,2580.0,510743.0,205.778640,505.146424,259.0
Huelva-Costa,4471.0,626.0,1514.0,288115.0,217.274352,525.484616,171.0
Huelva (capital),2319.0,327.0,738.0,143663.0,227.616018,513.702206,70.0
Condado-Campiña,2635.0,323.0,775.0,155057.0,208.310492,499.816197,68.0
Ayamonte,234.0,61.0,105.0,20946.0,291.225055,501.289029,28.0
Lepe,690.0,97.0,245.0,27431.0,353.614524,893.150086,25.0
Sierra de Huelva-Andévalo Central,1164.0,100.0,281.0,67571.0,147.992482,415.858874,19.0
Cartaya,254.0,43.0,117.0,19974.0,215.279864,585.761490,16.0
Palma del Condado (La),381.0,48.0,116.0,10761.0,446.055199,1077.966732,12.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,15.0,11.0,12.0,548.0,2007.299270,2189.781022,3.0
Villablanca,52.0,8.0,37.0,2848.0,280.898876,1299.157303,1.0
Palma del Condado (La),381.0,48.0,116.0,10761.0,446.055199,1077.966732,12.0
Villarrasa,87.0,4.0,21.0,2176.0,183.823529,965.073529,1.0
Lepe,690.0,97.0,245.0,27431.0,353.614524,893.150086,25.0
Aracena,278.0,11.0,68.0,8107.0,135.685210,838.781300,0.0
Lucena del Puerto,49.0,11.0,27.0,3371.0,326.312667,800.949273,3.0
Marines (Los),5.0,1.0,3.0,396.0,252.525253,757.575758,0.0
Campillo (El),51.0,5.0,15.0,2023.0,247.157687,741.473060,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,49.0,1.0,11.0,3509.0,28.498148,313.479624,0.0,0.090909
Villalba del Alcor,82.0,1.0,9.0,3338.0,29.958059,269.622528,0.0,0.111111
Aracena,278.0,11.0,68.0,8107.0,135.685210,838.781300,0.0,0.161765
Zarza-Perrunal (La),12.0,1.0,6.0,1267.0,78.926598,473.559590,0.0,0.166667
Villarrasa,87.0,4.0,21.0,2176.0,183.823529,965.073529,1.0,0.190476
Villablanca,52.0,8.0,37.0,2848.0,280.898876,1299.157303,1.0,0.216216
Valverde del Camino,252.0,17.0,69.0,12820.0,132.605304,538.221529,-2.0,0.246377
Galaroza,7.0,1.0,4.0,1379.0,72.516316,290.065265,0.0,0.250000
Punta Umbría,192.0,21.0,73.0,15242.0,137.777195,478.939772,5.0,0.287671
